In [3]:
import requests

url = "https://cricbuzz-cricket.p.rapidapi.com/matches/v1/recent"
headers = {
    "X-RapidAPI-Host": "cricbuzz-cricket.p.rapidapi.com",
    "X-RapidAPI-Key": "2cd68ffe5dmsh66a7797f04645a5p1837aajsncd1e5502a233"  # Use your personal key
}

response = requests.get(url, headers=headers)
data = response.json()

print(data)

# Display some match data (first 3 matches)
for series in data.get('typeMatches', []):
    if 'seriesMatches' in series:
        for ser in series['seriesMatches']:
            if 'seriesAdWrapper' in ser and 'matches' in ser['seriesAdWrapper']:
                for match in ser['seriesAdWrapper']['matches'][:3]:
                    info = match['matchInfo']
                    print(f"{info['matchDesc']}: {info['team1']['teamName']} vs {info['team2']['teamName']}")

{'typeMatches': [{'matchType': 'International', 'seriesMatches': [{'seriesAdWrapper': {'seriesId': 10587, 'seriesName': 'Asia Cup 2025', 'matches': [{'matchInfo': {'matchId': 130019, 'seriesId': 10587, 'seriesName': 'Asia Cup 2025', 'matchDesc': '2nd Match, Group A', 'matchFormat': 'T20', 'startDate': '1757514600000', 'endDate': '1757527200000', 'state': 'Complete', 'status': 'India won by 9 wkts', 'team1': {'teamId': 7, 'teamName': 'United Arab Emirates', 'teamSName': 'UAE', 'imageId': 172121}, 'team2': {'teamId': 2, 'teamName': 'India', 'teamSName': 'IND', 'imageId': 719031}, 'venueInfo': {'id': 153, 'ground': 'Dubai International Cricket Stadium', 'city': 'Dubai', 'timezone': '+04:00', 'latitude': '25.046842', 'longitude': '55.218969'}, 'currBatTeamId': 2, 'seriesStartDt': '1757289600000', 'seriesEndDt': '1759104000000', 'isTimeAnnounced': True, 'stateTitle': 'Complete'}, 'matchScore': {'team1Score': {'inngs1': {'inningsId': 1, 'runs': 57, 'wickets': 10, 'overs': 13.1}}, 'team2Score

In [4]:
matches_to_store = []

for series in data.get('typeMatches', []):
    if 'seriesMatches' in series:
        for ser in series['seriesMatches']:
            if 'seriesAdWrapper' in ser and 'matches' in ser['seriesAdWrapper']:
                for match in ser['seriesAdWrapper']['matches']:
                    info = match['matchInfo']
                    match_id = info.get('matchId')
                    desc = info.get('matchDesc')
                    date_epoch = info.get('startDate')
                    # convert epoch to date
                    from datetime import datetime
                    match_date = datetime.fromtimestamp(int(date_epoch) / 1000).date() if date_epoch else None

                    venue = info.get('venueInfo', {}).get('ground')
                    city = info.get('venueInfo', {}).get('city')
                    country = info.get('venueInfo', {}).get('country')
                    team1 = info.get('team1', {}).get('teamName')
                    team2 = info.get('team2', {}).get('teamName')
                    series_name = info.get('seriesName')

                    matches_to_store.append({
                        'match_id': match_id,
                        'description': desc,
                        'team1': team1,
                        'team2': team2,
                        'venue': venue,
                        'city': city,
                        'country': country,
                        'match_date': match_date,
                        'series_name': series_name
                    })
print(matches_to_store[:2])  # Preview first 2 matches to check

[{'match_id': 130019, 'description': '2nd Match, Group A', 'team1': 'United Arab Emirates', 'team2': 'India', 'venue': 'Dubai International Cricket Stadium', 'city': 'Dubai', 'country': None, 'match_date': datetime.date(2025, 9, 10), 'series_name': 'Asia Cup 2025'}, {'match_id': 130008, 'description': '1st Match, Group B', 'team1': 'Afghanistan', 'team2': 'Hong Kong', 'venue': 'Sheikh Zayed Stadium', 'city': 'Abu Dhabi', 'country': None, 'match_date': datetime.date(2025, 9, 9), 'series_name': 'Asia Cup 2025'}]


In [5]:
from utils.db_connection import engine
from utils.models import Base

Base.metadata.create_all(bind=engine)
print("All tables created (if not already present).")

All tables created (if not already present).


In [6]:
import sys
import os
sys.path.append(os.path.abspath('..'))

from utils.db_connection import SessionLocal
from utils.models import Match

session = SessionLocal()
for match in matches_to_store:
    # Avoid duplicates: check if match_id already exists
    if not session.query(Match).filter_by(description=match['description'], team1=match['team1'], team2=match['team2'], match_date=match['match_date']).first():
        m = Match(
            description=match['description'],
            team1=match['team1'],
            team2=match['team2'],
            venue=match['venue'],
            city=match['city'],
            country=match['country'],
            match_date=match['match_date']
        )
        session.add(m)
session.commit()
session.close()
print("Match data saved to your database.")

Match data saved to your database.


In [7]:
session = SessionLocal()
all_matches = session.query(Match).all()
for m in all_matches:
    print(m.description, m.team1, m.team2, m.match_date)
session.close()

3rd ODI Australia South Africa 2025-08-24
5th Match Hungary Croatia 2025-08-30
3rd Match Serbia Croatia 2025-08-30
1st Match Hungary Serbia 2025-08-29
2nd Match Croatia Hungary 2025-08-29
4th Match Hungary Serbia 2025-08-30
2nd T20I Switzerland Guernsey 2025-08-30
1st T20I Guernsey Switzerland 2025-08-30
2nd Match Pakistan United Arab Emirates 2025-08-30
1st Match Pakistan Afghanistan 2025-08-29
81st Match Namibia Scotland 2025-08-29
80th Match Canada Namibia 2025-08-27
1st T20I Netherlands Bangladesh 2025-08-30
4th T20I Belgium Austria 2025-08-24
3rd T20I Belgium Austria 2025-08-24
1st ODI Sri Lanka Zimbabwe 2025-08-29
12th Match Saint Lucia Kings Barbados Royals 2025-08-25
16th Match Barbados Royals Trinbago Knight Riders 2025-08-30
17th Match Guyana Amazon Warriors Trinbago Knight Riders 2025-08-31
15th Match St Kitts and Nevis Patriots Saint Lucia Kings 2025-08-29
13th Match Guyana Amazon Warriors Saint Lucia Kings 2025-08-27
11th Match St Kitts and Nevis Patriots Antigua and Barbu

In [8]:
from utils.db_connection import engine
from utils.models import Base  # Make sure Base includes Team class

Base.metadata.create_all(bind=engine)
print("Teams table created (if not present).")

Teams table created (if not present).


In [9]:
from utils.db_connection import engine
from utils.models import Base  # This should include your Team model

Base.metadata.create_all(bind=engine)
print("All tables, including Teams, created.")

All tables, including Teams, created.


In [10]:
import sys
import os
sys.path.append(os.path.abspath('..'))

from utils.db_connection import SessionLocal
from utils.models import Team

teams_to_store = []
for series in data.get('typeMatches', []):
    for ser in series.get('seriesMatches', []):
        for match in ser.get('seriesAdWrapper', {}).get('matches', []):
            for team_key in ['team1', 'team2']:
                team_info = match['matchInfo'].get(team_key)
                if team_info:
                    team_id = team_info.get('teamId')
                    team_name = team_info.get('teamName')
                    short_name = team_info.get('teamSName')
                    team_dict = {'team_id': team_id, 'team_name': team_name, 'short_name': short_name}
                    if team_dict not in teams_to_store:
                        teams_to_store.append(team_dict)

session = SessionLocal()
for t in teams_to_store:
    if not session.query(Team).filter_by(team_id=t['team_id']).first():
        team = Team(team_id=t['team_id'], team_name=t['team_name'], short_name=t['short_name'])
        session.add(team)
session.commit()
session.close()

print(f"{len(teams_to_store)} teams saved to the database.")

60 teams saved to the database.


In [11]:
from utils.db_connection import engine
from utils.models import Base

Base.metadata.create_all(bind=engine)
print("Players table created (if not present).")

Players table created (if not present).


In [12]:
import sys
import os
sys.path.append(os.path.abspath('..'))

from utils.db_connection import SessionLocal
from utils.models import Player

players_to_store = []

# Extract players from matches in the API data
for series in data.get('typeMatches', []):
    for ser in series.get('seriesMatches', []):
        for match in ser.get('seriesAdWrapper', {}).get('matches', []):
            # Players may be under 'players' or similar key, check the JSON structure
            players = match.get('players', {})
            for player_id, player_info in players.items():
                full_name = player_info.get('name')
                playing_role = player_info.get('role')
                batting_style = player_info.get('battingStyle')
                bowling_style = player_info.get('bowlingStyle')

                player_dict = {
                    'full_name': full_name,
                    'playing_role': playing_role,
                    'batting_style': batting_style,
                    'bowling_style': bowling_style
                }
                if player_dict not in players_to_store:
                    players_to_store.append(player_dict)

# Insert players into the database
session = SessionLocal()
for p in players_to_store:
    if not session.query(Player).filter_by(full_name=p['full_name']).first():
        player = Player(
            full_name=p['full_name'],
            playing_role=p['playing_role'],
            batting_style=p['batting_style'],
            bowling_style=p['bowling_style']
        )
        session.add(player)
session.commit()
session.close()

print(f"{len(players_to_store)} players saved to the database.")

0 players saved to the database.


In [ ]:
# Print sample from matches to check for player data
for series in data.get('typeMatches', []):
    for ser in series.get('seriesMatches', []):
        for match in ser.get('seriesAdWrapper', {}).get('matches', []):
            print("Match Description:", match['matchInfo'].get('matchDesc'))
            print("Keys in match:", match.keys())
            # Check if 'players' key exists and if yes, print a few player entries
            if 'players' in match:
                print("Players found:")
                player_samples = list(match.get('players').items())[:3]
                for pid, pinfo in player_samples:
                    print(pid, pinfo)
            else:
                print("No players key in this match")
            # Only check first match for brevity
            break
        break
    break

In [ ]:
from utils.db_connection import SessionLocal
from utils.models import Player

# Assuming 'data' variable holds your API JSON response which contains player info somewhere

players_to_store = []

# Adjust these loops/keys based on actual JSON structure of your 'data'
for series in data.get('typeMatches', []):
    for ser in series.get('seriesMatches', []):
        for match in ser.get('seriesAdWrapper', {}).get('matches', []):
            # Example: Find player data in the correct nested structure
            players = match.get('players', {})
            # If 'players' key not present here, try to find where player data resides in 'data'
            for player_id, player_info in players.items():
                full_name = player_info.get('name')
                playing_role = player_info.get('role')
                batting_style = player_info.get('battingStyle')
                bowling_style = player_info.get('bowlingStyle')
                player_dict = {
                    'full_name': full_name,
                    'playing_role': playing_role,
                    'batting_style': batting_style,
                    'bowling_style': bowling_style
                }
                if player_dict not in players_to_store:
                    players_to_store.append(player_dict)

# Insert players into the database
session = SessionLocal()
for p in players_to_store:
    if not session.query(Player).filter_by(full_name=p['full_name']).first():
        player = Player(
            full_name=p['full_name'],
            playing_role=p['playing_role'],
            batting_style=p['batting_style'],
            bowling_style=p['bowling_style']
        )
        session.add(player)
session.commit()
session.close()

print(f"{len(players_to_store)} players saved to the database.")

In [ ]:
import json
print(json.dumps(data, indent=2))

In [13]:
team_ids = [t['team_id'] for t in teams_to_store]


In [19]:
import requests
from utils.db_connection import SessionLocal
from utils.models import Player

players_to_store = []

for team_id in team_ids:
    url = f"https://cricbuzz-cricket.p.rapidapi.com/teams/v1/{team_id}"
    headers = {
        "X-RapidAPI-Host": "cricbuzz-cricket.p.rapidapi.com",
        "X-RapidAPI-Key": "2cd68ffe5dmsh66a7797f04645a5p1837aajsncd1e5502a233"  # your key
    }
    response = requests.get(url, headers=headers)
    data = response.json()
    for squad in data.get('squads', []):
        for player in squad.get('players', []):
            full_name = player.get('name')
            playing_role = player.get('playingRole')
            batting_style = player.get('battingStyle')
            bowling_style = player.get('bowlingStyle')
            player_dict = {
                'full_name': full_name,
                'playing_role': playing_role,
                'batting_style': batting_style,
                'bowling_style': bowling_style
            }
            if player_dict not in players_to_store:
                players_to_store.append(player_dict)

session = SessionLocal()
for p in players_to_store:
    if not session.query(Player).filter_by(full_name=p['full_name']).first():
        player = Player(
            full_name=p['full_name'],
            playing_role=p['playing_role'],
            batting_style=p['batting_style'],
            bowling_style=p['bowling_style']
        )
        session.add(player)
session.commit()
session.close()

print(f"{len(players_to_store)} players saved to the database.")

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [20]:
import requests

for team_id in team_ids:
    url = f"https://cricbuzz-cricket.p.rapidapi.com/teams/v1/{team_id}"
    headers = {
        "X-RapidAPI-Host": "cricbuzz-cricket.p.rapidapi.com",
        "X-RapidAPI-Key": "2cd68ffe5dmsh66a7797f04645a5p1837aajsncd1e5502a233"
    }
    response = requests.get(url, headers=headers)
    print(f"Team ID: {team_id}, Status Code: {response.status_code}")
    print("First 300 characters of response text:")
    print(response.text[:300])  # show a preview of the response

    # Only try to load JSON if status is 200
    if response.status_code == 200:
        try:
            data = response.json()
            print("Keys in response JSON:", data.keys())
            break  # after one successful fetch
        except Exception as e:
            print("JSON error:", e)
    else:
        print("Skipping due to non-200 response")
    break  # remove this to loop through all, keep it to test one

Team ID: 7, Status Code: 204
First 300 characters of response text:

Skipping due to non-200 response


In [21]:
import requests

for team_id in team_ids:
    url = f"https://cricbuzz-cricket.p.rapidapi.com/teams/v1/{team_id}"
    headers = {
        "X-RapidAPI-Host": "cricbuzz-cricket.p.rapidapi.com",
        "X-RapidAPI-Key": "2cd68ffe5dmsh66a7797f04645a5p1837aajsncd1e5502a233"
    }
    response = requests.get(url, headers=headers)
    print(f"Team ID: {team_id}, Status Code: {response.status_code}")
    print("First 300 characters of response text:")
    print(response.text[:300])
    print()  # Add a blank line for easier reading

Team ID: 7, Status Code: 204
First 300 characters of response text:


Team ID: 2, Status Code: 204
First 300 characters of response text:


Team ID: 96, Status Code: 204
First 300 characters of response text:


Team ID: 8, Status Code: 204
First 300 characters of response text:


Team ID: 11, Status Code: 204
First 300 characters of response text:


Team ID: 9, Status Code: 204
First 300 characters of response text:


Team ID: 3, Status Code: 204
First 300 characters of response text:


Team ID: 26, Status Code: 204
First 300 characters of response text:


Team ID: 23, Status Code: 204
First 300 characters of response text:


Team ID: 161, Status Code: 204
First 300 characters of response text:


Team ID: 12, Status Code: 204
First 300 characters of response text:


Team ID: 5, Status Code: 204
First 300 characters of response text:


Team ID: 557, Status Code: 204
First 300 characters of response text:


Team ID: 534, Status Code: 204
First 300 characters of response text:


Team ID: 

In [17]:
import requests

major_team_ids = [2, 4, 5, 9]  # India, Australia, South Africa, England
for team_id in major_team_ids:
    url = f"https://cricbuzz-cricket.p.rapidapi.com/teams/v1/{team_id}"
    headers = {
        "X-RapidAPI-Host": "cricbuzz-cricket.p.rapidapi.com",
        "X-RapidAPI-Key": "2cd68ffe5dmsh66a7797f04645a5p1837aajsncd1e5502a233"
    }
    response = requests.get(url, headers=headers)
    print(f"Team ID: {team_id}, Status Code: {response.status_code}")
    print("First 500 characters of response text:")
    print(response.text[:500])
    print()  # Blank line for readability

Team ID: 2, Status Code: 204
First 500 characters of response text:


Team ID: 4, Status Code: 204
First 500 characters of response text:


Team ID: 5, Status Code: 204
First 500 characters of response text:


Team ID: 9, Status Code: 204
First 500 characters of response text:




In [22]:
import requests
from bs4 import BeautifulSoup

url = "https://www.cricbuzz.com/cricket-team/india/2/players"
headers = {"User-Agent": "Mozilla/5.0"}
response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.content, "lxml")

players_to_store = []
# Find player blocks in each section on the page
for container in soup.find_all("div", class_="cb-col cb-col-100 cb-lst-itm cb-pos-rel cb-team-tp"):
    for player_item in container.find_all("a", class_="cb-text-link"):
        player_name = player_item.get_text(strip=True)
        player_url = "https://www.cricbuzz.com" + player_item["href"]
        players_to_store.append({"full_name": player_name, "profile_url": player_url})

print(players_to_store[:5])
print("Total players extracted:", len(players_to_store))

[]
Total players extracted: 0


In [2]:
import requests

def get_live_matches():
    url = "https://cricbuzz-cricket.p.rapidapi.com/matches/v1/live"
    headers = {
        "X-RapidAPI-Host": "cricbuzz-cricket.p.rapidapi.com",
        "X-RapidAPI-Key": "2cd68ffe5dmsh66a7797f04645a5p1837aajsncd1e5502a233"
    }
    response = requests.get(url, headers=headers)
    
    if response.status_code != 200:
        print("Failed to fetch live matches.")
        return []
    
    data = response.json()
    
    match_list = []
    for series in data.get('typeMatches', []):
        if 'seriesMatches' in series:
            for ser in series['seriesMatches']:
                if 'seriesAdWrapper' in ser and 'matches' in ser['seriesAdWrapper']:
                    for match in ser['seriesAdWrapper']['matches']:
                        info = match['matchInfo']
                        match_list.append({
                            'matchDesc': info['matchDesc'],
                            'team1': info['team1']['teamName'],
                            'team2': info['team2']['teamName'],
                            'status': info.get('status', 'Status Unavailable'),
                            'startTime': info.get('startTime', 'Start time unknown')
                        })
    return match_list

# Test print
matches = get_live_matches()
for m in matches:
    print(f"{m['matchDesc']}: {m['team1']} vs {m['team2']} - Status: {m['status']}")

Final: South Zone vs Central Zone - Status: Day 4: 1st Session - South Zone trail by 206 runs
3rd Match: Austria Women vs Switzerland Women - Status: Switzerland Women won by 6 wkts
2nd Match: Austria Women vs Luxembourg Women - Status: Austria Women won by 54 runs (DLS method) - Revised to 18 overs due to rain
4th Match: Trinbago Knight Riders Women vs Guyana Amazon Warriors Women - Status: Guyana Amazon Warriors Women won by 5 wkts
2nd Semi-Final: Vanuatu Women vs Japan Women - Status: Vanuatu Women won by 22 runs
1st Semi-Final: Indonesia Women vs Papua New Guinea Women - Status: Papua New Guinea Women won by 5 wkts
